In [88]:
import poliastro as pa

import astropy.units as u
from astropy import time

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit

from poliastro.plotting import OrbitPlotter3D

import numpy as np

In [89]:
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

<ScienceState solar_system_ephemeris: 'jpl'>

In [131]:
class Env():
    def __init__(self):
        
        self.nond_l = 1*u.AU
        self.nond_v = (Sun.k/self.nond_l)**0.5
        self.nond_t = self.nond_l/self.nond_v
        
        self.nond_m = 525.2*u.kg
        self.T_max = 2*u.N
        self.c = 2500*10*u.m/u.s**2
        
        
        
        self.t_ref = time.Time(2454396.30662312, format='jd') #used in linares paper
        
        
        
        self.accel_factor = (self.T_max*self.nond_t)/(self.nond_m*self.nond_v)
        self.mdot_factor = (self.T_max*self.nond_t)/(self.nond_m*self.c)
        
        self.dt = 14.38*u.day
    
    def get_final_loc(self, tof):
        
        date = self.t_ref + tof*self.nond_t
        
        ss_mars  = Orbit.from_body_ephem(Mars, date)
        
        r1, v1 = ss_mars.rv()
        
        r1 = (r1/self.nond_l).decompose().value
        v1 = (v1/self.nond_v).decompose().value
        
        return r1, v1
    
    def reset(self):
        
        date_launch = time.Time(2454396.30662312, format='jd') #used in linares paper
        
        self.t_ref = date_launch
        
        #todo: if we change the t_ref and date_launch, we need to update this.
        tof = 0. #((date_launch - self.t_ref)/self.nond_t).decompose().value #non dimensional flight time elapsed
        
        
        ss_earth = Orbit.from_body_ephem(Earth, date_launch)
        ss_mars  = Orbit.from_body_ephem(Mars,  date_launch)

        r0, v0 = ss_earth.rv()
        r1, v1 = ss_mars.rv()
        
        r0 = (r0/self.nond_l).decompose().value
        v0 = (v0/self.nond_v).decompose().value
        
        r1 = (r1/self.nond_l).decompose().value
        v1 = (v1/self.nond_v).decompose().value
        
        m = 1.
        
        state = np.array(r0)
        state = np.append(state, v0)
        state = np.append(state, m)
        state = np.append(state, r1-r0)
        state = np.append(state, v1-v0)
        state = np.append(state, tof)
        
        return state
    
    def step(self, state, action):
        
        r0 = state[:3]
        v0 = state[3:6]
        m  = state[6]
        rd = state[7:10]
        vd = state[10:13]
        tof = state[13]
        
        dt = self.dt
        
        # action represents the thrust in the three directions (non-dimensionalized)
        
        
        # ---- dynamics
        a = (thrust/m)*self.accel_factor
        
        r0dot = v0
        v0dot = a - r0*np.linalg.norm(r0)**-3
        mdot  = thrust*self.mdot_factor

        
        newr0 = r0 + r0dot*dt
    
        #todo
        
        

In [110]:
env = Env()

In [111]:
s = env.reset()

In [112]:
s

array([ 0.87176682,  0.4458208 ,  0.19318943, -0.50019317,  0.79956144,
        0.34660095,  1.        , -0.12686006,  0.7456648 ,  0.33300161,
       -0.17496311, -0.37519576, -0.13372221,  0.        ])

In [113]:
s[3:6]

array([-0.50019317,  0.79956144,  0.34660095])

In [117]:
T_max = 2*u.N

In [118]:
env.nond_m

<Quantity 525.2 kg>

In [119]:
(T_max*env.nond_t/(env.nond_m*env.nond_v)).decompose()

<Quantity 0.64216179>

In [71]:
(date_launch2 - date_launch).to(u.day).value

2.0277777777777777

In [123]:
r0 = s[:3]
r0

array([0.87176682, 0.4458208 , 0.19318943])

1.0059457080065208

In [70]:
date_launch = time.Time("2011-11-26 15:02", scale="utc")
date_launch2 = time.Time("2011-11-28 15:42", scale="utc")

In [31]:
ss_mars2 = Orbit.from_horizons('Mars Barycenter', epoch=date_launch, id_type='majorbody')

In [32]:
ss_mars2.r.to(u.AU)

<Quantity [-0.8198083 ,  1.27617303,  0.60748717] AU>

In [41]:
ss_earth = Orbit.from_body_ephem(Earth, date_launch)
ss_mars = Orbit.from_body_ephem(Mars, date_launch)

/Users/Devansh/anaconda3/lib/python3.6/site-packages/poliastro/twobody/orbit.py:389: TimeScaleWarning:

Input time was converted to scale='tdb' with value 2011-11-26 15:03. Use Time(..., scale='tdb') instead.



In [42]:
ss_mars

1 x 2 AU x 24.7 deg (ICRS) orbit around Sun (☉) at epoch 2011-11-26 15:03 (TDB)

In [35]:
ss_earth.plane

<Planes.EARTH_EQUATOR: 'Earth mean Equator and Equinox of epoch (J2000.0)'>

In [37]:
plotter = OrbitPlotter3D()
plotter.plot(ss_mars2, label="Mars2", color="red")
plotter.plot(ss_earth, label="Earth", color="navy")
plotter.plot(ss_mars, label="Mars", color="red")



FigureWidget({
    'data': [{'line': {'color': 'red', 'dash': 'dash', 'width': 5},
              'mode': 'line…

In [38]:
ss_mars.r.to(u.AU)

<Quantity [-0.82320444,  1.27510178,  0.60702505] AU>

In [39]:
ss_mars2.r.to(u.AU)

<Quantity [-0.8198083 ,  1.27617303,  0.60748717] AU>

In [40]:
ss_mars.to_icrs()

1 x 2 AU x 24.7 deg (ICRS) orbit around Sun (☉) at epoch 2011-11-26 15:03 (TDB)